# Analisis de superficial del csv de clicks
### En este notebook exploramos los registros del csv con el objetivo de buscar relaciones entre las distintas variables

In [1]:
import xgboost as xgb
from sklearn.metrics import mean_squared_error
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

## Leemos el csv, y dropeamos las columnas action_id por estar llena de nans y wifi_connection por estar llena de False

In [2]:
clicks = pd.read_csv("../data/clicks.csv", dtype={'advertiser_id': 'category', 'action_id': 'float64', 'source_id': 'category',
                                               'created': 'str', 'country_code':'category', 'latitude': 'float64',
                                               'longitude':'float64', 'wifi_connection':'bool' , 'carrier_id':np.float16,
                                               'trans_id':'str', 'os_minor':'float64', 'agent_device':'float64', 'os_major':'float64',
                                               'specs_brand':'int64', 'brand':'category', 'timeToClick':'float64',
                                               'ref_type':'category', 'ref_hash':'int64'}, parse_dates = ['created'])
clicks.drop('action_id', axis=1, inplace = True)
clicks.drop('wifi_connection', axis=1, inplace = True) #false
clicks['day'] = clicks['created'].dt.date
clicks['hour'] = clicks['created'].dt.hour
clicks.head(5)

D:\Users\Lucas\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3049: DtypeWarning: Columns (16,17) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,advertiser_id,source_id,created,country_code,latitude,longitude,carrier_id,trans_id,os_minor,agent_device,os_major,specs_brand,brand,timeToClick,touchX,touchY,ref_type,ref_hash,day,hour
0,1,2,2019-04-18 05:27:42.197000+00:00,6287817205707153877,1.714547,0.871535,3.0,9JMAfrb-b9cSEVCJb0P9JfihGthaS7E,1.517644e+18,NaN,5.131616e+18,71913840936116953,0.0,2.317,0.968,0.503,1891515180541284343,1293710398598742392,2019-04-18,5
1,1,1,2019-04-18 05:27:03.164000+00:00,6287817205707153877,1.714512,0.871062,2.0,r3xtTRv2lInfiXG8JI3NQsNcBo8GyFQ,1.288578e+18,NaN,3.908390e+18,3576558787748411622,1.0,7.653,0.712,1.689,1891515180541284343,1663930990551616564,2019-04-18,5
2,1,1,2019-04-18 05:42:07.926000+00:00,6287817205707153877,1.714547,0.871535,4.0,WOnHFqQtY48z_ygKZ-030U_g0TMGVMw,2.238736e+18,NaN,3.581233e+18,3576558787748411622,NaN,464.796,0.227,0.251,1891515180541284343,8488038938665586188,2019-04-18,5
3,1,1,2019-04-18 05:26:04.446000+00:00,6287817205707153877,1.708041,0.870772,1.0,wQMLLmYqiFhSuha9p9B13PMtcyBW_vM,2.411640e+18,NaN,3.908390e+18,3576558787748411622,NaN,225.311,0.696,6.587,1891515180541284343,6488361690105189959,2019-04-18,5
4,1,1,2019-04-18 05:23:37.764000+00:00,6287817205707153877,1.715514,0.870772,2.0,GeFoyBzMA7taylMxxjzlNPTU-n4FXFs,1.517644e+18,NaN,5.131616e+18,3576558787748411622,0.0,84.736,0.059,0.142,1891515180541284343,1348993302102753419,2019-04-18,5


In [3]:
clicks=clicks.drop(['source_id','advertiser_id','country_code','carrier_id','trans_id','os_minor','agent_device','os_major','specs_brand','ref_type'],axis=1)
clicks['created'].max()

Timestamp('2019-04-26 23:59:22.065000+0000', tz='UTC')

In [4]:
clicks['created'].min()

Timestamp('2019-04-12 00:00:01.981000+0000', tz='UTC')

In [5]:
clicks.dtypes

created        datetime64[ns, UTC]
latitude                   float64
longitude                  float64
brand                     category
timeToClick                float64
touchX                      object
touchY                      object
ref_hash                     int64
day                         object
hour                         int64
dtype: object

## Realizamos un histograma del TimeToClick

In [6]:
#clicks_acotado = clicks.dropna(subset= ['timeToClick'])

## Creamos una columna categorica del TimeToClick

In [7]:
#bins = pd.IntervalIndex.from_tuples([(600, 120000), (180, 600), (60, 180), (15, 60), (3, 15), (0, 3)])
#clicks_acotado['time_categ'] = pd.cut(clicks_acotado['timeToClick'],bins)
#clicks_acotado['time_categ'].cat.categories = ['10 a 2000min','3 a 10min','1 a 3 min','15 a 60seg','3 a 15seg','0 a 3 seg']

#clicks['time_categ'] = pd.cut(clicks['timeToClick'],bins)
#clicks['time_categ'].cat.categories = ['10 a 2000min','3 a 10min','1 a 3 min','15 a 60seg', '3 a 15seg','0 a 3 seg']

#clicks_acotado['time_categ'].value_counts()

## Elegimos las columnas con menor entropia o pocos valores diferentes y nos quedamos con los 5 valores mas comunes

In [8]:
grupos = ['hour', 'day']
#top5 = {}
#for i in grupos:
#    top5[i]= clicks[i].value_counts().head(5).index.tolist()
#top5

In [9]:
clicks = clicks.sort_values(by = 'created')
clicks = clicks.sort_values(by = 'ref_hash')
clicks['repeticiones'] = 1
device_ids = clicks['ref_hash'].to_frame().set_index('ref_hash')
#clicks = pd.get_dummies(clicks, columns = grupos)
#for column in grupos:
#    for value in top5[column]:
#        clicks[column + str(value)] = (clicks[column] == value)*1

In [10]:
fecha_minima=pd.to_datetime('2019-04-18 00:00:00', utc = True)
fecha_minima
fecha_tope=pd.to_datetime('2019-04-21 00:00:00', utc = True)
clicks_ventana1=clicks.loc[((clicks['created']>fecha_minima) & (clicks['created']<fecha_tope) )].copy(deep=False)

In [11]:
fecha_minima=pd.to_datetime('2019-04-19 00:00:00', utc = True)
fecha_minima
fecha_tope=pd.to_datetime('2019-04-22 00:00:00', utc = True)
clicks_ventana2=clicks.loc[((clicks['created']>fecha_minima) & (clicks['created']<fecha_tope) )].copy(deep=False)

In [12]:
fecha_minima=pd.to_datetime('2019-04-20 00:00:00', utc = True)
fecha_minima
fecha_tope=pd.to_datetime('2019-04-23 00:00:00', utc = True)
clicks_ventana3=clicks.loc[((clicks['created']>fecha_minima) & (clicks['created']<fecha_tope) )].copy(deep=False)

In [13]:
fecha_minima=pd.to_datetime('2019-04-21 00:00:00', utc = True)
fecha_minima
fecha_tope=pd.to_datetime('2019-04-24 00:00:00', utc = True)
clicks_ventana4=clicks.loc[((clicks['created']>fecha_minima) & (clicks['created']<fecha_tope) )].copy(deep=False)

In [14]:
fecha_minima=pd.to_datetime('2019-04-24 00:00:00', utc = True)
fecha_minima
fecha_tope=pd.to_datetime('2019-04-27 00:00:00', utc = True)
clicks_ventana7=clicks.loc[((clicks['created']>fecha_minima) & (clicks['created']<fecha_tope) )].copy(deep=False)

In [15]:
features = ['sum', 'mean', 'std', 'min', 'max']
ventanas = [clicks_ventana1, clicks_ventana2, clicks_ventana3, clicks_ventana4, clicks_ventana7]
numero_de_ventanas = [1,2,3,4,7]
contador = 0
for ventana in ventanas:
    features_ventana = ventana.groupby('ref_hash').agg(features).fillna(0)
    level0 = features_ventana.columns.get_level_values(0)
    level1 = features_ventana.columns.get_level_values(1)
    features_ventana.columns = level0 + "_" + level1
    features_ventana.to_csv('../xgb/features_clicks_lucas_ventana' + str(numero_de_ventanas[contador]) + '.csv')
    contador = contador + 1